# Part 4: Buid a Mode Choice Model
## Choose Model and Estimated Parameters
    Build a discrete choice models for at least 2 OD pairs for 2022 data.



In [64]:
#import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.formula.api import logit
import statsmodels.api as sm
import scipy as sp
import geopandas as gpd
from shapely import wkt

In [65]:
#### Store the url string that hosts our .csv files 
#### Copy data files to the 2022CMS folder that should be in the same folder as this notebook
urlday = "C:\\Users\\Administrator\\Downloads\\project_data\\Citywide_Mobility_Survey_-_Day_2022.csv"
urlhhl = "C:\\Users\\Administrator\\Downloads\\project_data\\Citywide_Mobility_Survey_-_Household_2022.csv"
urlper = "C:\\Users\\Administrator\\Downloads\\project_data\\Citywide_Mobility_Survey_-_Person_2022.csv"
urlzon = "C:\\Users\\Administrator\\Downloads\\project_data\\Citywide_Mobility_Survey_-_Survey_Zones.csv"
urltrp = "C:\\Users\\Administrator\\Downloads\\project_data\\Citywide_Mobility_Survey_-_Trip_2022.csv"
urlveh = "C:\\Users\\Administrator\\Downloads\\project_data\\Citywide_Mobility_Survey_-_Vehicle_2022.csv"

#### Read the .csv file using `pandas` libriary and store it as a `pandas` Data Frame
dday = pd.read_csv(urlday)
dhhl = pd.read_csv(urlhhl)
dper = pd.read_csv(urlper)
dzon = pd.read_csv(urlzon)
dtrp = pd.read_csv(urltrp)
dveh = pd.read_csv(urlveh)

#### Finally, let's make sure that all columns are displayed
pd.set_option('display.max_columns', None)

In [66]:
zone_nums = list(range(12))
zone_names = {k: v for k, v in enumerate(dzon['CMS Zone'])}

dtrp['o_cms_zone_name'] = dtrp['o_cms_zone'].map(zone_names)
dtrp['d_cms_zone_name'] = dtrp['d_cms_zone'].map(zone_names)

tripzone_d = dtrp.groupby('d_cms_zone_name').size()
tripzone_o = dtrp.groupby('o_cms_zone_name').size()

all_zones = set(zone_names.values())
o_zones_with_data = set(tripzone_o.index)
d_zones_with_data = set(tripzone_d.index)

missing_origins = all_zones - o_zones_with_data
missing_destinations = all_zones - d_zones_with_data

tripzone_o = tripzone_o.reindex(all_zones, fill_value=0)
tripzone_d = tripzone_d.reindex(all_zones, fill_value=0)

tripzone = pd.crosstab(dtrp['o_cms_zone_name'], dtrp['d_cms_zone_name'])
tripzone = tripzone.reindex(index=all_zones, columns=all_zones, fill_value=0)

In [67]:
dth = pd.merge(dtrp,dhhl, how = 'outer', on='hh_id')
dthp = pd.merge(dth,dper, how = 'outer', on=['hh_id','person_id'])
dthp.describe()

,hh_id,person_id,person_num_x,day_id,travel_dow,day_num,is_complete,linked_trip_id,linked_leg_num,trip_id,trip_num,day_is_complete,arrive_dow,depart_dow,o_in_region,o_county,d_in_region,d_county,mode_type_nyc,r_mode_type_nyc,mode_1,mode_2,mode_3,mode_priority_nyc,transit_access,r_transit_access,transit_egress,r_transit_egress,park_location,park_type,park_pay,park_cost,ev_charge_station,ev_charge_station_level_1,ev_charge_station_level_2,ev_charge_station_level_3,ev_charge_station_level_4,ev_charge_station_level_5,ev_charge_station_level_997,ev_charge_station_level_998,tnc_type,taxi_type,taxi_pay,taxi_cost,transit_type,bike_park_loc,scooter_park_location,num_travelers,num_hh_travelers,num_non_hh_travelers,hh_member_1,hh_member_2,hh_member_3,hh_member_4,hh_member_5,hh_member_6,hh_member_7,hh_member_8,hh_member_9,hh_member_10,hh_member_11,hh_member_12,driver,o_purpose,o_purpose_category,d_purpose,d_purpose_category,d_purpose_category_linked,r_d_purpose_category_linked,work_trip,depart_hour,depart_minute,depart_seconds,r_timeofday,arrive_hour,arrive_minute,arrive_second,duration_minutes,r_duration_minutes,duration_seconds,distance_miles,r_distance_miles,dwell_mins,high_speed_flag,long_distance_flag,o_cms_zone,d_cms_zone,unlinked_trip,o_location_type,d_location_type,trip_wkday_weight,trip_svnday_weight,hh_cms_zone_gp,participation_group,num_days_complete_x,num_days_complete_weekday,num_days_complete_weekend,num_complete_mon,num_complete_tue,num_complete_wed,num_complete_thu,num_complete_fri,num_complete_sat,num_complete_sun,num_people,r_num_people,num_adults,num_kids,num_students,num_workers,num_vehicles,r_num_vehicles,num_trips_x,income_broad,primary_language,residence_type,residence_rent_own,home_county,vehicle_change,reduced_veh_1,reduced_veh_2,reduced_veh_3,reduced_veh_4,reduced_veh_5,reduced_veh_6,reduced_veh_7,reduced_veh_8,reduced_veh_997,increased_veh_1,increased_veh_2,increased_veh_3,increased_veh_4,increased_veh_5,increased_veh_6,increased_veh_7,increased_veh_8,increased_veh_997,ev_typical_charge_1,ev_typical_charge_2,ev_typical_charge_3,ev_typical_charge_4,ev_typical_charge_5,ev_typical_charge_6,ev_typical_charge_7,ev_typical_charge_997,ev_purchase,ev_purchase_barriers_1,ev_purchase_barriers_2,ev_purchase_barriers_3,ev_purchase_barriers_4,ev_purchase_barriers_5,num_bicycles,bicycle_type_1,bicycle_type_2,bicycle_type_997,bike_change,reduced_bike_1,reduced_bike_2,reduced_bike_3,reduced_bike_4,reduced_bike_5,reduced_bike_6,reduced_bike_7,reduced_bike_8,reduced_bike_997,increased_bike_1,increased_bike_2,increased_bike_3,increased_bike_4,increased_bike_5,increased_bike_6,increased_bike_7,increased_bike_8,increased_bike_9,increased_bike_997,bike_store_1,bike_store_2,bike_store_3,bike_store_4,bike_store_5,bike_store_6,bike_store_8,bike_store_9,bike_store_997,num_scooters,scooter_typical_charge_1,scooter_typical_charge_2,scooter_typical_charge_4,scooter_typical_charge_5,scooter_typical_charge_6,scooter_typical_charge_7,scooter_typical_charge_997,packages,hh_weight,num_trips_y,num_days_complete_y,person_num_y,is_participant,phone_type,relationship,age,r_age,race_1,race_2,race_3,race_4,race_5,race_6,race_997,race_999,r_race,ethnicity_1,ethnicity_2,ethnicity_3,ethnicity_4,ethnicity_5,ethnicity_997,ethnicity_999,r_ethnicity,gender,can_drive,disability,disability_follow_1,disability_follow_2,disability_follow_3,disability_follow_4,disability_follow_5,disability_follow_6,disability_follow_996,disability_follow_999,education,employment,worker,num_jobs,job_type,industry,work_in_region,work_county,work_cms_zone,wfh_policy,commute_freq,telework_freq,r_telework_freq,work_mode,r_work_mode,work_mode_own,work_mode_auto,work_park_loc,work_park_pay,work_park_cost,work_park_amount,work_mode_taxi,work_mode_bus,work_mode_rail,work_mode_ferry,work_mode_bike,work_bike_park,work_mode_scooter,work_scooter_park,student,r_student,school_type,r_school_type,school_in_region,school_county,school_freq,remote_class_freq,school_mode,school_mode_own,school_mod

In [68]:
dthp = dthp[dthp['r_mode_type_nyc'] != 995]
mode_name = {1: 'Vehicle', 2: 'Bus', 3: 'Rail', 4: 'Walk', 5: 'Bike', 6: 'Other'}
dthp_mode = dthp.groupby('r_mode_type_nyc').size()
dthp_mode.index = dthp_mode.index.map(mode_name)

dthp_mode

r_mode_type_nyc
Vehicle    23014
Bus         4456
Rail       15944
Walk       36992
Bike        1958
Other       1102
dtype: int64

In [69]:
dthp['slow_speed'] = 0
dthp.loc[:,'slow_speed'] = np.where((dthp['r_mode_type_nyc'] == 4) | (dthp['r_mode_type_nyc'] == 5), 1, 0)
dthp['slow_speed'].value_counts()

slow_speed
0    48746
1    38950
Name: count, dtype: int64

In [70]:
dthp['pub_transit'] = 0
dthp.loc[:,'pub_transit'] = np.where((dthp['r_mode_type_nyc'] == 2) | (dthp['r_mode_type_nyc'] == 3), 1 ,0)
dthp['pub_transit'].value_counts()

pub_transit
0    67296
1    20400
Name: count, dtype: int64

In [71]:
dthp['private'] = 0
dthp.loc[:,'private'] = np.where((dthp['r_mode_type_nyc'] == 1) | (dthp['r_mode_type_nyc'] == 6), 1 ,0)
dthp['private'].value_counts()

private
0    63580
1    24116
Name: count, dtype: int64

In [ ]:
dthp['income_lower'] = 0
dthp['income_higher'] = 0
dthp['education_higher'] = 0
dthp['Age_teenager'] = False
dthp['Age_elder'] = False
dthp['Gender'] = 0
dthp['HH_size'] = 0
dthp['driver_license'] = 0
dthp['vehicle_ava'] = 0

In [ ]:
# choose two specific origin-destination pairs to display (LGA to Upper Manhattan, LGA to LGA, and Outer Brooklyn to LGA)
dtrpz4o = dthp[dthp['o_cms_zone_name'] == 'LGA']
dtrpz3o = dthp[dthp['o_cms_zone_name'] == 'Outer Brooklyn']

dtrpz428 = dtrpz4o[dtrpz4o['d_cms_zone_name'] == 'Upper Manhattan']
dtrpz424 = dtrpz4o[dtrpz4o['d_cms_zone_name'] == 'LGA']
dtrpz324 = dtrpz3o[dtrpz3o['d_cms_zone_name'] == 'LGA']

display(dtrpz428)
display(dtrpz424)
display(dtrpz324)

,hh_id,person_id,person_num,day_id,travel_dow,day_num,is_complete,travel_date,linked_trip_id,linked_leg_num,trip_id,trip_num,day_is_complete,arrive_dow,depart_dow,o_in_region,o_county,d_in_region,d_county,mode_type_nyc,r_mode_type_nyc,mode_1,mode_2,mode_3,mode_priority_nyc,transit_access,r_transit_access,transit_egress,r_transit_egress,park_location,park_type,park_pay,park_cost,ev_charge_station,ev_charge_station_level_1,ev_charge_station_level_2,ev_charge_station_level_3,ev_charge_station_level_4,ev_charge_station_level_5,ev_charge_station_level_997,ev_charge_station_level_998,tnc_type,taxi_type,taxi_pay,taxi_cost,transit_type,bike_park_loc,scooter_park_location,num_travelers,num_hh_travelers,num_non_hh_travelers,hh_member_1,hh_member_2,hh_member_3,hh_member_4,hh_member_5,hh_member_6,hh_member_7,hh_member_8,hh_member_9,hh_member_10,hh_member_11,hh_member_12,driver,o_purpose,o_purpose_category,d_purpose,d_purpose_category,d_purpose_category_linked,r_d_purpose_category_linked,work_trip,depart_date,depart_hour,depart_minute,depart_seconds,r_timeofday,arrive_date,arrive_hour,arrive_minute,arrive_second,duration_minutes,r_duration_minutes,duration_seconds,distance_miles,r_distance_miles,dwell_mins,high_speed_flag,long_distance_flag,o_cms_zone,d_cms_zone,unlinked_trip,o_location_type,d_location_type,trip_wkday_weight,trip_svnday_weight,o_cms_zone_name,d_cms_zone_name,slow_speed,pub_transit,private
343,22000768,2200076801,1,220007680104,6,4,1,10/08/2022,2200076801035,4,2200076801037,41,1,6,6,1,36061,1,36047,6,3,30,1,995,30,1,1,1,1,995,995,995,NaN,995,995,995,995,995,995,995,995,995,995,995,NaN,10,995,995,1,1,0,1,0,0,995,995,995,995,995,995,995,995,995,995,60,11,60,11,2,2,1,10/08/2022,15,9,43,3,10/08/2022,15,23,3,13,2,800,6.420600,4,2.933333,0,0,4,8,1,4,4,0.0,40.752203,LGA,Upper Manhattan,0,1,0
355,22000768,2200076801,1,220007680105,7,5,1,10/09/2022,2200076801046,4,2200076801048,53,0,7,7,1,36061,1,36047,6,3,30,1,995,30,1,1,1,1,995,995,995,NaN,995,995,995,995,995,995,995,995,995,995,995,NaN,10,995,995,1,1,0,1,0,0,995,995,995,995,995,995,995,995,995,995,60,11,60,11,2,2,1,10/09/2022,15,1,25,3,10/09/2022,15,6,32,5,1,307,2.150734,3,0.000000,0,0,4,8,1,4,4,0.0,0.000000,LGA,Upper Manhattan,0,1,0
369,22000768,2200076801,1,220007680106,1,6,1,10/10/2022,2200076801059,4,2200076801062,67,0,1,1,1,36061,1,36047,6,3,30,1,995,30,1,1,1,1,995,995,995,NaN,995,995,995,995,995,995,995,995,995,995,995,NaN,10,995,995,1,1,0,1,0,0,995,995,995,995,995,995,995,995,995,995,60,11,60,11,2,2,1,10/10/2022,14,59,23,2,10/10/2022,15,4,46,5,1,323,2.154429,3,0.000000,0,0,4,8,1,4,4,0.0,0.000000,LGA,Upper Manhattan,0,1,0
957,22002589,2200258901,1,220025890104,5,4,1,10/07/2022,2200258901011,3,2200258901012,14,1,5,5,1,36061,1,36047,6,3,30,1,995,30,1,1,1,1,995,995,995,NaN,995,995,995,995,995,995,995,995,995,995,995,NaN,10,995,995,1,1,0,1,0,0,0,0,995,995,995,995,995,995,995,995,60,11,60,11,2,2,1,10/07/2022,7,35,3,1,10/07/2022,7,56,49,22,3,1306,3.539774,3,0.000000,0,0,4,8,1,4,4,0.0,422.056921,LGA,Upper Manhattan,0,1,0
1308,22003296,2200329601,1,220032960107,5,7,1,11/04/2022,2200329601039,3,2200329601040,42,0,5,5,1,36061,1,36047,9,4,1,995,995,1,995,995,995,995,995,995,995,NaN,995,995,995,995,995,995,995,995,995,995,995,NaN,995,995,995,2,1,1,1,995,995,995,995,995,995,995,995,995,995,995,995,60,11,60,11,11,11,0,11/04/2022,8,16,0,1,11/04/2022,8,23,58,8,1,478,3.101674,3,17.066667,1,0,4,8,0,4,4,0.0,0.000000,LGA,Upper Manhattan,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85928,22145707,2214570701,1,221457070107,2,7,1,10/04/2022,2214570701050,1,2214570701050,52,0,2,2,1,36061,1,36047,8,5,2,995,995,2,995,995,995,995,995,995,995,

,hh_id,person_id,person_num,day_id,travel_dow,day_num,is_complete,travel_date,linked_trip_id,linked_leg_num,trip_id,trip_num,day_is_complete,arrive_dow,depart_dow,o_in_region,o_county,d_in_region,d_county,mode_type_nyc,r_mode_type_nyc,mode_1,mode_2,mode_3,mode_priority_nyc,transit_access,r_transit_access,transit_egress,r_transit_egress,park_location,park_type,park_pay,park_cost,ev_charge_station,ev_charge_station_level_1,ev_charge_station_level_2,ev_charge_station_level_3,ev_charge_station_level_4,ev_charge_station_level_5,ev_charge_station_level_997,ev_charge_station_level_998,tnc_type,taxi_type,taxi_pay,taxi_cost,transit_type,bike_park_loc,scooter_park_location,num_travelers,num_hh_travelers,num_non_hh_travelers,hh_member_1,hh_member_2,hh_member_3,hh_member_4,hh_member_5,hh_member_6,hh_member_7,hh_member_8,hh_member_9,hh_member_10,hh_member_11,hh_member_12,driver,o_purpose,o_purpose_category,d_purpose,d_purpose_category,d_purpose_category_linked,r_d_purpose_category_linked,work_trip,depart_date,depart_hour,depart_minute,depart_seconds,r_timeofday,arrive_date,arrive_hour,arrive_minute,arrive_second,duration_minutes,r_duration_minutes,duration_seconds,distance_miles,r_distance_miles,dwell_mins,high_speed_flag,long_distance_flag,o_cms_zone,d_cms_zone,unlinked_trip,o_location_type,d_location_type,trip_wkday_weight,trip_svnday_weight,o_cms_zone_name,d_cms_zone_name,slow_speed,pub_transit,private
203,22000533,2200053301,1,220005330107,2,7,1,10/11/2022,2200053301037,3,2200053301039,42,1,2,2,1,36061,1,36061,9,4,995,995,995,1,1,1,1,1,995,995,995,NaN,995,995,995,995,995,995,995,995,995,995,995,NaN,10,995,995,1,1,0,1,0,995,995,995,995,995,995,995,995,995,995,995,60,11,60,11,1,1,0,10/11/2022,18,29,12,4,10/11/2022,19,56,27,87,4,5235,0.547374,1,5.433333,0,0,4,4,1,4,4,479.857858,274.204491,LGA,LGA,1,0,0
207,22000607,2200060701,1,220006070101,5,1,1,10/14/2022,2200060701002,1,2200060701002,2,1,5,5,1,36061,1,36061,9,4,1,995,995,1,995,995,995,995,995,995,995,NaN,995,995,995,995,995,995,995,995,995,995,995,NaN,995,995,995,1,1,0,1,0,0,995,995,995,995,995,995,995,995,995,995,60,11,30,7,7,7,0,10/14/2022,10,31,5,2,10/14/2022,10,42,39,12,2,694,0.204435,1,16.683333,0,0,4,4,0,4,2,0.000000,906.930375,LGA,LGA,1,0,0
208,22000607,2200060701,1,220006070101,5,1,1,10/14/2022,2200060701003,1,2200060701003,3,1,5,5,1,36061,1,36061,9,4,1,995,995,1,995,995,995,995,995,995,995,NaN,995,995,995,995,995,995,995,995,995,995,995,NaN,995,995,995,1,1,0,1,0,0,995,995,995,995,995,995,995,995,995,995,30,7,50,8,8,8,0,10/14/2022,10,59,20,2,10/14/2022,11,4,56,6,1,336,0.313033,1,77.000000,0,0,4,4,0,2,2,0.000000,906.930375,LGA,LGA,1,0,0
209,22000607,2200060701,1,220006070101,5,1,1,10/14/2022,2200060701004,1,2200060701004,4,1,5,5,1,36061,1,36061,9,4,1,995,995,1,995,995,995,995,995,995,995,NaN,995,995,995,995,995,995,995,995,995,995,995,NaN,995,995,995,1,1,0,1,0,0,995,995,995,995,995,995,995,995,995,995,50,8,11,3,3,3,0,10/14/2022,12,21,56,2,10/14/2022,12,27,19,5,1,323,0.167522,1,53.633333,0,0,4,4,0,2,4,0.000000,906.930375,LGA,LGA,1,0,0
210,22000607,2200060701,1,220006070101,5,1,1,10/14/2022,2200060701005,1,2200060701005,5,1,5,5,1,36061,1,36061,9,4,1,995,995,1,995,995,995,995,995,995,995,NaN,995,995,995,995,995,995,995,995,995,995,995,NaN,995,995,995,1,1,0,1,0,0,995,995,995,995,995,995,995,995,995,995,11,3,11,3,3,3,0,10/14/2022,13,20,57,2,10/14/2022,13,28,40,8,1,463,0.247722,1,8.483333,0,0,4,4,0,4,4,0.000000,906.930375,LGA,LGA,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86164,22145717,2214571701,1,221457170101,3,1,1,09/28/2022,2214571701003,1,2214571701003,4,1,3,3,1,36061,1,36061,9,4,1,995,995,1,995,995,995,995,995,995,995

,hh_id,person_id,person_num,day_id,travel_dow,day_num,is_complete,travel_date,linked_trip_id,linked_leg_num,trip_id,trip_num,day_is_complete,arrive_dow,depart_dow,o_in_region,o_county,d_in_region,d_county,mode_type_nyc,r_mode_type_nyc,mode_1,mode_2,mode_3,mode_priority_nyc,transit_access,r_transit_access,transit_egress,r_transit_egress,park_location,park_type,park_pay,park_cost,ev_charge_station,ev_charge_station_level_1,ev_charge_station_level_2,ev_charge_station_level_3,ev_charge_station_level_4,ev_charge_station_level_5,ev_charge_station_level_997,ev_charge_station_level_998,tnc_type,taxi_type,taxi_pay,taxi_cost,transit_type,bike_park_loc,scooter_park_location,num_travelers,num_hh_travelers,num_non_hh_travelers,hh_member_1,hh_member_2,hh_member_3,hh_member_4,hh_member_5,hh_member_6,hh_member_7,hh_member_8,hh_member_9,hh_member_10,hh_member_11,hh_member_12,driver,o_purpose,o_purpose_category,d_purpose,d_purpose_category,d_purpose_category_linked,r_d_purpose_category_linked,work_trip,depart_date,depart_hour,depart_minute,depart_seconds,r_timeofday,arrive_date,arrive_hour,arrive_minute,arrive_second,duration_minutes,r_duration_minutes,duration_seconds,distance_miles,r_distance_miles,dwell_mins,high_speed_flag,long_distance_flag,o_cms_zone,d_cms_zone,unlinked_trip,o_location_type,d_location_type,trip_wkday_weight,trip_svnday_weight,o_cms_zone_name,d_cms_zone_name,slow_speed,pub_transit,private
7,22000105,2200010501,1,220001050101,1,1,1,10/17/2022,2200010501004,1,2200010501004,4,1,1,1,1,36061,1,36061,9,4,995,995,995,1,3,0,5,0,995,995,995,NaN,995,995,995,995,995,995,995,995,995,995,995,NaN,10,995,995,1,1,0,1,0,995,995,995,995,995,995,995,995,995,995,2,60,11,60,11,11,11,0,10/17/2022,15,15,49,3,10/17/2022,15,19,31,4,1,222,0.246971,1,0.000000,0,0,3,4,1,4,4,861.630290,492.360166,Outer Brooklyn,LGA,1,0,0
58,22000105,2200010501,1,220001050106,6,6,1,10/22/2022,2200010501052,1,2200010501052,54,1,6,6,1,36061,1,36061,9,4,1,995,995,1,995,995,995,995,995,995,995,NaN,995,995,995,995,995,995,995,995,995,995,995,NaN,995,995,995,1,1,0,1,0,995,995,995,995,995,995,995,995,995,995,2,50,8,32,7,7,7,0,10/22/2022,12,12,30,2,10/22/2022,12,12,34,0,1,4,0.298171,1,8.983333,1,0,3,4,0,4,4,0.000000,492.360166,Outer Brooklyn,LGA,1,0,0
498,22001486,2200148601,1,220014860101,2,1,995,10/11/2022,2200148601002,1,2200148601005,4,1,2,2,1,36061,1,36061,6,3,30,995,995,30,1,1,1,1,995,995,995,NaN,995,995,995,995,995,995,995,995,995,995,995,NaN,10,995,995,1,1,0,1,0,995,995,995,995,995,995,995,995,995,995,995,11,3,60,11,3,3,0,10/11/2022,9,0,0,2,10/11/2022,9,0,0,0,995,0,NaN,995,0.000000,0,0,3,4,0,995,995,0.000000,0.000000,Outer Brooklyn,LGA,0,1,0
605,22001486,2200148601,1,220014860101,2,1,1,10/11/2022,2200148601002,2,2200148601002,5,1,2,2,1,36061,1,36061,6,3,30,995,995,30,1,1,1,1,995,995,995,NaN,995,995,995,995,995,995,995,995,995,995,995,NaN,10,995,995,1,1,0,1,0,995,995,995,995,995,995,995,995,995,995,995,60,11,60,11,3,3,0,10/11/2022,9,0,0,2,10/11/2022,9,50,0,50,3,3000,8.790817,4,0.000000,0,0,3,4,0,995,995,1199.483572,1199.483572,Outer Brooklyn,LGA,0,1,0
606,22001486,2200148601,1,220014860101,2,1,995,10/11/2022,2200148601002,3,2200148601008,6,1,2,2,1,36061,1,36061,6,3,30,995,995,30,1,1,1,1,995,995,995,NaN,995,995,995,995,995,995,995,995,995,995,995,NaN,10,995,995,1,1,0,1,0,995,995,995,995,995,995,995,995,995,995,995,60,11,11,3,3,3,0,10/11/2022,9,50,0,2,10/11/2022,9,50,0,0,995,0,NaN,995,310.000000,0,0,3,4,0,995,995,0.000000,0.000000,Outer Brooklyn,LGA,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85661,22145699,2214569901,1,221456990104,6,4,1,10/01/2022,2214569901015,2,2214569901016,18,1,6,6,1,36061,1,36061,6,3,30,1,

## Discuss Resulted Model Validity
    Only reasonable models should be included in the report. To increase the likelihood to generate the model with a good fit, use OD-pairs with largest number of predicted trips. 


## Mode Choice Model Results Discussions
    Discuss R-squired, intercept value, signs and significance of estimated parameters. 
    Discuss multiple noticeable observations based on estimated parameters: what unexpected significant factors you have found? Is there a difference in the models for different evaluated OD-pairs? 
